In [7]:
 # Import necessary libraries
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Lambda, Dot, Reshape, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
import random

In [8]:
 # Sample dataset: For demonstration, using a small text. Replace with a larger corpus as needed.
sample_text = """
In the age of information, data is the new oil. The ability to process and analyze data
has become a crucial skill in various industries. Machine learning and artificial intelligence
are driving innovations that were once thought impossible.
"""

In [9]:
# Initialize and fit the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts([sample_text])
word2id = tokenizer.word_index
id2word = {v: k for k, v in word2id.items()}
vocab_size = len(word2id) + 1  # +1 for padding

In [5]:
# Define window size
window_size = 2

In [6]:
# Generate skip-grams pairs
sequences = tokenizer.texts_to_sequences([sample_text])[0]
pairs, labels = skipgrams(sequences, vocabulary_size=vocab_size, window_size=window_size, shuffle=True)


In [10]:
# Separate target and context words
target_words, context_words = zip(*pairs)
target_words = np.array(target_words, dtype='int32')
context_words = np.array(context_words, dtype='int32')
labels = np.array(labels, dtype='int32')

In [11]:
# Stage c: Train the model
embedding_dim = 50

In [12]:
# Input for target words
target_input = Input(shape=(1,), name='target_input')
# Input for context words
context_input = Input(shape=(1,), name='context_input')

In [13]:
# Embedding layer
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, name='embedding_layer')

In [23]:
# Embed target and context words
target_embedding = embedding(target_input)
context_embedding = embedding(context_input)

In [24]:
# Compute dot product between target and context embeddings
dot_product = Dot(axes=-1)([target_embedding, context_embedding])
dot_product = Reshape((1,))(dot_product)

In [25]:
# Define the output
output = Dense(1, activation='sigmoid')(dot_product)

In [26]:
# Define and compile the model
cbow_model = Model(inputs=[target_input, context_input], outputs=output)
cbow_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

ValueError: Argument(s) not recognized: {'lr': 0.001}

In [27]:
# Train the model
cbow_model.fit([target_words, context_words], labels, epochs=10, batch_size=64, verbose=2)

ValueError: You must call `compile()` before using the model.

In [20]:
# Extract the embeddings from the trained model
word_embeddings = cbow_model.get_layer('embedding_layer').get_weights()[0]

IndentationError: unexpected indent (Temp/ipykernel_13664/1904174560.py, line 2)

In [28]:
# Function to get the embedding of a word
def get_embedding(word):
    if word in word2id:
        return word_embeddings[word2id[word]]
    else:
        return np.zeros(embedding_dim)

In [29]:
# Example: Get embeddings for some words
words = ['data', 'machine', 'learning', 'information', 'innovation']
for word in words:
    emb = get_embedding(word)
    print(f'Embedding for "{word}":\n{emb}\n')

NameError: name 'word_embeddings' is not defined